In [65]:
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)

# Re-import modified modules without restarting the server
%load_ext autoreload
%autoreload 2
%load_ext dotenv
%dotenv


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [66]:
from attribution.api_attribution import OpenAIAttributor
from attribution.experiment_logger import ExperimentLogger
from attribution.token_perturbation import (
    FixedPerturbationStrategy,
)

attributor = OpenAIAttributor(request_chunksize=10)

In [67]:
kwargs = {
    "perturbation_strategy": FixedPerturbationStrategy(""),
    "attribution_strategies": ["cosine"],  
    "perturb_word_wise": True,
    "ignore_output_token_location": True,
}

In [68]:
input_str = "Mary puts an apple in the box. The box is labelled 'pencils'. John enters the room. What does he think is in the box? Answer in 1 word."
response = await attributor.get_chat_completion(input_str)
print(response.message.content)

Apples


In [69]:
logger = ExperimentLogger()
await attributor.compute_attributions(
    input_str,
    logger=logger,
    **kwargs
)
display(logger.df_experiments)
logger.print_sentence_attribution()
logger.print_attribution_matrix(exp_id=1, show_debug_cols=True)

Sending 10 concurrent requests at a time: 100%|██████████| 3/3 [00:01<00:00,  1.57it/s]


,exp_id,original_input,original_output,perturbation_strategy,perturb_word_wise,duration,num_llm_calls
0,1,Mary puts an apple in the box. The box is labe...,Apples,fixed,True,2.406061,28


,exp_id,attribution_strategy,perturbation_strategy,perturb_word_wise,token_1,token_2,token_3,token_4,token_5,token_6,token_7,token_8,token_9,token_10,token_11,token_12,token_13,token_14,token_15,token_16,token_17,token_18,token_19,token_20,token_21,token_22,token_23,token_24,token_25,token_26,token_27,token_28
0,1,cosine,fixed,True,Mary 0.68,puts 0.69,an 0.00,apple 0.69,in 0.00,the 0.00,box. 0.00,The 0.00,box 0.00,is 0.00,labelled 0.00,'pencils'. 0.68,John 0.00,enters 0.00,the 0.00,room. 0.00,What 0.00,does 0.00,he 0.00,think 0.68,is 0.00,in 0.69,the 0.00,box? 0.00,Answer 0.00,in 0.00,1 0.69,word. 0.69


Attribution matrix for experiment 1 
Attribution Strategy: cosine 
Perturbation strategy: fixed:
Input Tokens (Rows) vs. Output Tokens (Columns)


,App (0),les (1),perturbed_input,perturbed_output
Mary (0),0.583758,0.776083,puts an apple in the box. The box is labelled 'pencils'. John enters the room. What does he think is in the box? Answer in 1 word.,Apple
puts (1),0.669369,0.712692,Mary an apple in the box. The box is labelled 'pencils'. John enters the room. What does he think is in the box? Answer in 1 word.,Pencils
an (2),0.000000,0.000000,Mary puts apple in the box. The box is labelled 'pencils'. John enters the room. What does he think is in the box? Answer in 1 word.,Apples
apple (3),0.669369,0.712692,Mary puts an in the box. The box is labelled 'pencils'. John enters the room. What does he think is in the box? Answer in 1 word.,Pencils
in (4),0.000000,0.000000,Mary puts an apple the box. The box is labelled 'pencils'. John enters the room. What does he think is in the box? Answer in 1 word.,Apples
the (5),0.000000,0.000000,Mary puts an apple in box. The box is labelled 'pencils'. John enters the room. What does he think is in the box? Answer in 1 word.,Apples
box. (6),0.000000,0.000000,Mary puts an apple in the The box is labelled 'pencils'. John enters the room. What does he think is in the box? Answer in 1 word.,Apples
The (7),0.000000,0.000000,Mary puts an apple in the box. box is labelled 'pencils'. John enters the room. What does he think is in the box? Answer in 1 word.,Apples
box (8),0.000000,0.000000,Mary puts an apple in the box. The is labelled 'pencils'. John enters the room. What does he think is in the box? Answer in 1 word.,Apples
is (9),0.000000,0.000000,Mary puts an apple in the box. The box labelled 'pencils'. John enters the room. What does he think is in the box? Answer in 1 word.,Apples


In [70]:
logger = ExperimentLogger()
await attributor.hierarchical_perturbation(
    input_str,
    init_chunksize=8,
    stages=4,
    logger=logger,
    **kwargs
)
display(logger.df_experiments)
logger.print_sentence_attribution()
logger.print_attribution_matrix(exp_id=1, show_debug_cols=True)

processing chunk:  box is labelled 'pencils'. John enters the room. What does he think is in the box? Answer in 1 word.
processing chunk: Mary puts an apple in the box. The What does he think is in the box? Answer in 1 word.
processing chunk: Mary puts an apple in the box. The box is labelled 'pencils'. John enters the room. Answer in 1 word.
processing chunk: Mary puts an apple in the box. The box is labelled 'pencils'. John enters the room. What does he think is in the box?
processing chunk:  in the box. The box is labelled 'pencils'. John enters the room. What does he think is in the box? Answer in 1 word.
processing chunk: Mary puts an apple box is labelled 'pencils'. John enters the room. What does he think is in the box? Answer in 1 word.
processing chunk: Mary puts an apple in the box. The John enters the room. What does he think is in the box? Answer in 1 word.
processing chunk: Mary puts an apple in the box. The box is labelled 'pencils'. What does he think is in the box? Answ

Sending 10 concurrent requests at a time: 100%|██████████| 2/2 [00:01<00:00,  1.39it/s]


processing chunk:  puts an apple in the box. The box is labelled 'pencils'. John enters the room. What does he think is in the box? Answer in 1 word.
processing chunk: Mary an apple in the box. The box is labelled 'pencils'. John enters the room. What does he think is in the box? Answer in 1 word.
processing chunk: Mary puts apple in the box. The box is labelled 'pencils'. John enters the room. What does he think is in the box? Answer in 1 word.
processing chunk: Mary puts an in the box. The box is labelled 'pencils'. John enters the room. What does he think is in the box? Answer in 1 word.
processing chunk: Mary puts an apple the box. The box is labelled 'pencils'. John enters the room. What does he think is in the box? Answer in 1 word.
processing chunk: Mary puts an apple in box. The box is labelled 'pencils'. John enters the room. What does he think is in the box? Answer in 1 word.
processing chunk: Mary puts an apple in the box. The box is 'pencils'. John enters the room. What doe

Sending 10 concurrent requests at a time: 100%|██████████| 2/2 [00:01<00:00,  1.50it/s]


,exp_id,original_input,original_output,perturbation_strategy,perturb_word_wise,duration,num_llm_calls
0,1,Mary puts an apple in the box. The box is labe...,Apples,fixed,True,7.181204,45


,exp_id,attribution_strategy,perturbation_strategy,perturb_word_wise,token_1,token_2,token_3,token_4,token_5,token_6,token_7,token_8,token_9,token_10,token_11,token_12,token_13,token_14,token_15,token_16,token_17,token_18,token_19,token_20,token_21,token_22,token_23,token_24,token_25,token_26,token_27,token_28
0,1,cosine,fixed,True,Mary 0.00,puts 0.69,an 0.00,apple 0.69,in 0.00,the 0.00,box. 0.00,The 0.00,box 0.00,is 0.00,labelled 0.00,'pencils'. 0.68,John 0.00,enters 0.00,the 0.00,room. 0.00,What 0.00,does 0.00,he 0.00,think 0.68,is 0.00,in 0.69,the 0.00,box? 0.00,Answer 0.00,in 0.69,1 0.70,word. 0.70


Attribution matrix for experiment 1 
Attribution Strategy: cosine 
Perturbation strategy: fixed:
Input Tokens (Rows) vs. Output Tokens (Columns)


,App (0),les (1),perturbed_input,perturbed_output
Mary (0),0.000000,0.000000,puts an apple in the box. The box is labelled 'pencils'. John enters the room. What does he think is in the box? Answer in 1 word.,Apples
puts (1),0.669369,0.712692,Mary an apple in the box. The box is labelled 'pencils'. John enters the room. What does he think is in the box? Answer in 1 word.,Pencils
an (2),0.000000,0.000000,Mary puts apple in the box. The box is labelled 'pencils'. John enters the room. What does he think is in the box? Answer in 1 word.,Apples
apple (3),0.669369,0.712692,Mary puts an in the box. The box is labelled 'pencils'. John enters the room. What does he think is in the box? Answer in 1 word.,Pencils
in (4),0.000000,0.000000,Mary puts an apple the box. The box is labelled 'pencils'. John enters the room. What does he think is in the box? Answer in 1 word.,Apples
the (5),0.000000,0.000000,Mary puts an apple in box. The box is labelled 'pencils'. John enters the room. What does he think is in the box? Answer in 1 word.,Apples
box. (6),0.000000,0.000000,Mary puts an apple in the box is labelled 'pencils'. John enters the room. What does he think is in the box? Answer in 1 word.,Apples
The (7),0.000000,0.000000,Mary puts an apple in the box is labelled 'pencils'. John enters the room. What does he think is in the box? Answer in 1 word.,Apples
box (8),0.000000,0.000000,Mary puts an apple in the box. The labelled 'pencils'. John enters the room. What does he think is in the box? Answer in 1 word.,Apples
is (9),0.000000,0.000000,Mary puts an apple in the box. The labelled 'pencils'. John enters the room. What does he think is in the box? Answer in 1 word.,Apples


In [87]:
short_prompts = [
    "What is the capital of the country that was formerly known as Siam? Answer in 1 word.",
    "What is the chemical element with the symbol 'Au'? Answer in 1 word.",
    "Which character in 'Pride and Prejudice' said 'It is a truth universally acknowledged'? Answer in 1 word.",
    "Who holds the record for the most goals in a calendar year in football (soccer)? Answer in 1 word.",
    "In Greek mythology, who is the goddess of wisdom and warfare? Answer in 1 word.",
    "Who directed the film 'Inception'? Answer in 1 word.",
    "Who is known as the 'King of Pop'? Answer in 1 word.",
    "Who was the first female Prime Minister of the United Kingdom? Answer in 1 word.",
    "Who is the co-founder of Microsoft? Answer in 1 word.",
    "Who painted the 'Mona Lisa'? Answer in 1 word.",
]

for input_str in short_prompts:
    response = await attributor.get_chat_completion(input_str)

    logger = ExperimentLogger()

    await attributor.compute_attributions(
        input_str,
        logger=logger,
        **kwargs
    )
    # logger.print_sentence_attribution()
    # logger.print_attribution_matrix(exp_id=1, attribution_strategy="cosine")
    
    await attributor.hierarchical_perturbation(
        input_str,
        init_chunksize=8,
        logger=logger,
        **kwargs
    )

    logger.print_sentence_attribution()
    # logger.print_attribution_matrix(exp_id=2, attribution_strategy="cosine")
    display(logger.df_experiments)

Sending 10 concurrent requests at a time: 100%|██████████| 2/2 [00:01<00:00,  1.45it/s]


,exp_id,attribution_strategy,perturbation_strategy,perturb_word_wise,token_1,token_2,token_3,token_4,token_5,token_6,token_7,token_8,token_9,token_10,token_11,token_12,token_13,token_14,token_15,token_16,token_17
0,1,cosine,fixed,True,What 0.00,is 0.00,the 0.00,capital 0.59,of 0.00,the 0.00,country 0.00,that 0.00,was 0.00,formerly 0.00,known 0.00,as 0.00,Siam? 0.64,Answer 0.00,in 0.00,1 0.00,word. 0.00
1,2,cosine,fixed,True,What 0.00,is 0.00,the 0.00,capital 0.59,of 0.00,the 0.00,country 0.00,that 0.00,was 0.00,formerly 0.00,known 0.00,as 0.00,Siam? 0.66,Answer 0.00,in 0.00,1 0.00,word. 0.00


,exp_id,original_input,original_output,perturbation_strategy,perturb_word_wise,duration,num_llm_calls
0,1,What is the capital of the country that was fo...,Bangkok,fixed,True,1.678971,17
1,2,What is the capital of the country that was fo...,Bangkok,fixed,True,5.58223,16


Sending 10 concurrent requests at a time: 100%|██████████| 2/2 [00:01<00:00,  1.52it/s]


,exp_id,attribution_strategy,perturbation_strategy,perturb_word_wise,token_1,token_2,token_3,token_4,token_5,token_6,token_7,token_8,token_9,token_10,token_11,token_12,token_13
0,1,cosine,fixed,True,What 0.00,is 0.00,the 0.00,chemical 0.00,element 0.00,with 0.00,the 0.00,symbol 0.00,'Au'? 0.73,Answer 0.00,in 0.00,1 0.00,word. 0.00
1,2,cosine,fixed,True,What 0.00,is 0.00,the 0.00,chemical 0.00,element 0.00,with 0.00,the 0.00,symbol 0.00,'Au'? 0.64,Answer 0.00,in 0.00,1 0.00,word. 0.00


...

,exp_id,original_input,original_output,perturbation_strategy,perturb_word_wise,duration,num_llm_calls
0,1,Who was the first female Prime Minister of the...,Margaret,fixed,True,1.376653,15
1,2,Who was the first female Prime Minister of the...,Margaret,fixed,True,4.23889,21


,exp_id,attribution_strategy,perturbation_strategy,perturb_word_wise,token_1,token_2,token_3,token_4,token_5,token_6,token_7,token_8,token_9,token_10
0,1,cosine,fixed,True,Who 0.00,is 0.00,the 0.00,co-founder 0.64,of 0.00,Microsoft? 0.50,Answer 0.00,in 0.00,1 -0.00,word. -0.00
1,2,cosine,fixed,True,Who 0.00,is 0.00,the 0.00,co-founder 0.00,of 0.00,Microsoft? 0.51,Answer 0.00,in 0.00,1 -0.00,word. -0.00


,exp_id,original_input,original_output,perturbation_strategy,perturb_word_wise,duration,num_llm_calls
0,1,Who is the co-founder of Microsoft? Answer in ...,Bill,fixed,True,1.354034,10
1,2,Who is the co-founder of Microsoft? Answer in ...,Bill,fixed,True,2.160125,11


,exp_id,attribution_strategy,perturbation_strategy,perturb_word_wise,token_1,token_2,token_3,token_4,token_5,token_6,token_7,token_8,token_9
0,1,cosine,fixed,True,Who 0.00,painted 0.75,the 0.00,'Mona 0.00,Lisa'? 0.69,Answer 0.00,in 0.20,1 0.14,word. 0.14
1,2,cosine,fixed,True,Who 0.00,painted 0.00,the 0.00,'Mona 0.00,Lisa'? 0.14,Answer 0.14,in 0.14,1 0.14,word. 0.14


,exp_id,original_input,original_output,perturbation_strategy,perturb_word_wise,duration,num_llm_calls
0,1,Who painted the 'Mona Lisa'? Answer in 1 word.,Da Vinci,fixed,True,1.130457,9
1,2,Who painted the 'Mona Lisa'? Answer in 1 word.,Da Vinci,fixed,True,3.03571,9


In [89]:
longer_prompts = [
    "In J.K. Rowling's Harry Potter series, the spell used to conjure a Patronus is considered highly advanced and can only be performed by skilled witches and wizards. The form that a Patronus takes can vary widely and is often influenced by the caster's personality and experiences. What form does Snape's Patronus take? Answer in 1 word.",
    "During the American Civil War, there was a significant battle fought from July 1 to July 3, 1863, which is often considered the turning point of the war. This battle took place in Pennsylvania and ended with a decisive victory for the Union forces. What is the name of this battle? Answer in 1 word.",
    "In the field of astronomy, there is a phenomenon where the light from a star is bent and magnified by the gravitational field of another object, such as a galaxy or black hole, that lies between the star and the observer. This effect was first predicted by Einstein's theory of general relativity. What is this phenomenon called? Answer in 1 word.",
    "Located in South America, there is a vast river that flows through Brazil, Peru, and several other countries. It is the largest river by discharge volume of water in the world and is often associated with the rainforest of the same name. What is the name of this river? Answer in 1 word.",
    "In Norse mythology, there is a hammer wielded by the god Thor, which is renowned for its immense power and is said to be capable of leveling mountains. This hammer is also a symbol of protection and blessing. What is the name of Thor's hammer? Answer in 1 word.",
    "In the movie 'The Matrix,' the protagonist is a computer hacker who learns about the true nature of his reality and his role in the war against its controllers. He is given a choice between two pills: a red pill that reveals the truth, and a blue pill that returns him to his normal life. What is the name of the protagonist? Answer in 1 word.",
    "There is a famous painting by Vincent van Gogh that depicts a night sky filled with swirling clouds, stars, and a bright crescent moon. This painting is one of his most well-known works and was created while he was in a mental asylum in Saint-Rémy-de-Provence. What is the title of this painting? Answer in 1 word.",
    "In classical music, there is a composer who is renowned for his symphonies, concertos, and sonatas. Born in Salzburg in 1756, he began composing music at a very young age and created over 600 works during his lifetime. What is the last name of this composer? Answer in 1 word.",
    "In the realm of computer programming, there is a widely used language that was developed by Guido van Rossum and first released in 1991. It emphasizes code readability and its syntax allows programmers to express concepts in fewer lines of code. What is the name of this programming language? Answer in 1 word.",
    "In the study of genetics, there is a molecule that carries the genetic instructions used in the growth, development, functioning, and reproduction of all known living organisms and many viruses. This molecule is structured as a double helix and was first described by Watson and Crick in 1953. What is the abbreviation for this molecule? Answer in 1 word.",
]

for input_str in longer_prompts:
    response = await attributor.get_chat_completion(input_str)
    print(response.message.content)

    logger = ExperimentLogger()

    await attributor.compute_attributions(
        input_str,
        logger=logger,
        **kwargs
    )

    await attributor.hierarchical_perturbation(
        input_str,
        init_chunksize=8,
        logger=logger,
        **kwargs
    )

    logger.print_sentence_attribution()
    display(logger.df_experiments)

Doe


Sending 10 concurrent requests at a time: 100%|██████████| 6/6 [00:02<00:00,  2.29it/s]


,exp_id,attribution_strategy,perturbation_strategy,perturb_word_wise,token_1,token_2,token_3,token_4,token_5,token_6,token_7,token_8,token_9,token_10,token_11,token_12,token_13,token_14,token_15,token_16,token_17,token_18,token_19,token_20,token_21,token_22,token_23,token_24,token_25,token_26,token_27,token_28,token_29,token_30,token_31,token_32,token_33,token_34,token_35,token_36,token_37,token_38,token_39,token_40,token_41,token_42,token_43,token_44,token_45,token_46,token_47,token_48,token_49,token_50,token_51,token_52,token_53,token_54,token_55,token_56
0,1,cosine,fixed,True,In -0.00,J.K. -0.00,Rowling's -0.00,Harry -0.00,Potter -0.00,"series, -0.00",the -0.00,spell -0.00,used -0.00,to -0.00,conjure -0.00,a -0.00,Patronus -0.00,is -0.00,considered -0.00,highly -0.00,advanced -0.00,and -0.00,can -0.00,only -0.00,be -0.00,performed -0.00,by -0.00,skilled -0.00,witches -0.00,and -0.00,wizards. -0.00,The -0.00,form -0.00,that -0.00,a -0.00,Patronus -0.00,takes -0.00,can -0.00,vary -0.00,widely -0.00,and -0.00,is -0.00,often -0.00,influenced -0.00,by -0.00,the -0.00,caster's -0.00,personality -0.00,and -0.00,experiences. -0.00,What -0.00,form -0.00,does -0.00,Snape's 0.66,Patronus -0.00,take? -0.00,Answer -0.00,in -0.00,1 -0.00,word. 0.59
1,2,cosine,fixed,True,In -0.00,J.K. -0.00,Rowling's -0.00,Harry -0.00,Potter -0.00,"series, -0.00",the -0.00,spell -0.00,used -0.00,to -0.00,conjure -0.00,a -0.00,Patronus -0.00,is -0.00,considered -0.00,highly -0.00,advanced -0.00,and -0.00,can -0.00,only -0.00,be -0.00,performed -0.00,by -0.00,skilled -0.00,witches -0.00,and -0.00,wizards. -0.00,The -0.00,form -0.00,that -0.00,a -0.00,Patronus -0.00,takes -0.00,can -0.00,vary -0.00,widely -0.00,and -0.00,is -0.00,often -0.00,influenced -0.00,by -0.00,the -0.00,caster's -0.00,personality -0.00,and -0.00,experiences. -0.00,What -0.00,form -0.00,does -0.00,Snape's 0.66,Patronus -0.00,take? -0.00,Answer -0.00,in -0.00,1 -0.00,word. 0.56


,exp_id,original_input,original_output,perturbation_strategy,perturb_word_wise,duration,num_llm_calls
0,1,"In J.K. Rowling's Harry Potter series, the spe...",Doe,fixed,True,3.684383,56
1,2,"In J.K. Rowling's Harry Potter series, the spe...",Doe,fixed,True,11.911259,18


Gettysburg


...

,exp_id,original_input,original_output,perturbation_strategy,perturb_word_wise,duration,num_llm_calls
0,1,"In the realm of computer programming, there is...",Python,fixed,True,3.615485,53
1,2,"In the realm of computer programming, there is...",Python,fixed,True,2.696447,12


DNA


Sending 10 concurrent requests at a time: 100%|██████████| 6/6 [00:02<00:00,  2.30it/s]


,exp_id,attribution_strategy,perturbation_strategy,perturb_word_wise,token_1,token_2,token_3,token_4,token_5,token_6,token_7,token_8,token_9,token_10,token_11,token_12,token_13,token_14,token_15,token_16,token_17,token_18,token_19,token_20,token_21,token_22,token_23,token_24,token_25,token_26,token_27,token_28,token_29,token_30,token_31,token_32,token_33,token_34,token_35,token_36,token_37,token_38,token_39,token_40,token_41,token_42,token_43,token_44,token_45,token_46,token_47,token_48,token_49,token_50,token_51,token_52,token_53,token_54,token_55,token_56,token_57,token_58,token_59
0,1,cosine,fixed,True,In -0.00,the -0.00,study -0.00,of -0.00,"genetics, -0.00",there -0.00,is -0.00,a -0.00,molecule -0.00,that -0.00,carries -0.00,the -0.00,genetic -0.00,instructions -0.00,used -0.00,in -0.00,the -0.00,"growth, -0.00","development, -0.00","functioning, -0.00",and -0.00,reproduction -0.00,of -0.00,all -0.00,known -0.00,living -0.00,organisms -0.00,and -0.00,many -0.00,viruses. -0.00,This -0.00,molecule -0.00,is -0.00,structured -0.00,as -0.00,a -0.00,double -0.00,helix -0.00,and -0.00,was -0.00,first -0.00,described -0.00,by -0.00,Watson -0.00,and -0.00,Crick -0.00,in -0.00,1953. -0.00,What -0.00,is -0.00,the -0.00,abbreviation -0.00,for -0.00,this -0.00,molecule? -0.00,Answer -0.00,in -0.00,1 -0.00,word. -0.00
1,2,cosine,fixed,True,In -0.00,the -0.00,study -0.00,of -0.00,"genetics, -0.00",there -0.00,is -0.00,a -0.00,molecule -0.00,that -0.00,carries -0.00,the -0.00,genetic -0.00,instructions -0.00,used -0.00,in -0.00,the -0.00,"growth, -0.00","development, -0.00","functioning, -0.00",and -0.00,reproduction -0.00,of -0.00,all -0.00,known -0.00,living -0.00,organisms -0.00,and -0.00,many -0.00,viruses. -0.00,This -0.00,molecule -0.00,is -0.00,structured -0.00,as -0.00,a -0.00,double -0.00,helix -0.00,and -0.00,was -0.00,first -0.00,described -0.00,by -0.00,Watson -0.00,and -0.00,Crick -0.00,in -0.00,1953. -0.00,What -0.00,is -0.00,the -0.00,abbreviation -0.00,for -0.00,this -0.00,molecule? -0.00,Answer -0.00,in -0.00,1 -0.00,word. -0.00


,exp_id,original_input,original_output,perturbation_strategy,perturb_word_wise,duration,num_llm_calls
0,1,"In the study of genetics, there is a molecule ...",DNA,fixed,True,3.461527,59
1,2,"In the study of genetics, there is a molecule ...",DNA,fixed,True,2.549622,11


In [86]:
longer_prompts = [
    # "In J.K. Rowling's Harry Potter series, the spell used to conjure a Patronus is considered highly advanced and can only be performed by skilled witches and wizards. The form that a Patronus takes can vary widely and is often influenced by the caster's personality and experiences. What form does Snape's Patronus take? Answer in 1 word.",
    # "In J.K. Rowling's Harry Potter series, the spell used to conjure a Patronus is considered highly advanced and can only be performed by skilled witches and wizards. The form that a Patronus takes can vary widely and is often influenced by the caster's personality and experiences. What form does Severus Snape's Patronus take? Answer in 1 word.",
    "In J.K. Rowling's Harry Potter series, the spell used to conjure a Patronus is considered highly advanced and can only be performed by skilled witches and wizards. The form that a Patronus takes can vary widely and is often influenced by the caster's personality and experiences. What form does Malfoy's Patronus take? Answer in 1 word.",
]

for input_str in longer_prompts:
    response = await attributor.get_chat_completion(input_str)
    print(response.message.content)

    logger = ExperimentLogger()

    await attributor.compute_attributions(
        input_str,
        logger=logger,
        **kwargs
    )

    await attributor.hierarchical_perturbation(
        input_str,
        init_chunksize=8,
        logger=logger,
        **kwargs
    )

    logger.print_sentence_attribution()
    display(logger.df_experiments)

Snake


Sending 10 concurrent requests at a time: 100%|██████████| 6/6 [00:07<00:00,  1.26s/it]


,exp_id,attribution_strategy,perturbation_strategy,perturb_word_wise,token_1,token_2,token_3,token_4,token_5,token_6,token_7,token_8,token_9,token_10,token_11,token_12,token_13,token_14,token_15,token_16,token_17,token_18,token_19,token_20,token_21,token_22,token_23,token_24,token_25,token_26,token_27,token_28,token_29,token_30,token_31,token_32,token_33,token_34,token_35,token_36,token_37,token_38,token_39,token_40,token_41,token_42,token_43,token_44,token_45,token_46,token_47,token_48,token_49,token_50,token_51,token_52,token_53,token_54,token_55,token_56
0,1,cosine,fixed,True,In -0.00,J.K. -0.00,Rowling's -0.00,Harry -0.00,Potter -0.00,"series, -0.00",the -0.00,spell -0.00,used -0.00,to -0.00,conjure -0.00,a -0.00,Patronus -0.00,is -0.00,considered -0.00,highly -0.00,advanced -0.00,and -0.00,can -0.00,only -0.00,be -0.00,performed -0.00,by -0.00,skilled -0.00,witches -0.00,and -0.00,wizards. -0.00,The -0.00,form -0.00,that -0.00,a 0.74,Patronus -0.00,takes -0.00,can -0.00,vary -0.00,widely -0.00,and -0.00,is 0.66,often -0.00,influenced -0.00,by -0.00,the -0.00,caster's -0.00,personality 0.74,and -0.00,experiences. -0.00,What 0.65,form -0.00,does -0.00,Malfoy's 0.67,Patronus 0.74,take? -0.00,Answer -0.00,in 0.66,1 0.68,word. 0.68
1,2,cosine,fixed,True,In -0.00,J.K. -0.00,Rowling's -0.00,Harry -0.00,Potter -0.00,"series, -0.00",the -0.00,spell -0.00,used -0.00,to -0.00,conjure -0.00,a -0.00,Patronus -0.00,is -0.00,considered -0.00,highly -0.00,advanced -0.00,and -0.00,can -0.00,only -0.00,be -0.00,performed -0.00,by -0.00,skilled -0.00,witches -0.00,and -0.00,wizards. -0.00,The -0.00,form -0.00,that -0.00,a -0.00,Patronus -0.00,takes -0.00,can -0.00,vary -0.00,widely -0.00,and -0.00,is -0.00,often -0.00,influenced -0.00,by -0.00,the -0.00,caster's -0.00,personality -0.00,and -0.00,experiences. -0.00,What -0.00,form -0.00,does -0.00,Malfoy's 0.67,Patronus -0.00,take? -0.00,Answer -0.00,in -0.00,1 0.35,word. 0.68


,exp_id,original_input,original_output,perturbation_strategy,perturb_word_wise,duration,num_llm_calls
0,1,"In J.K. Rowling's Harry Potter series, the spe...",Snake,fixed,True,8.395655,56
1,2,"In J.K. Rowling's Harry Potter series, the spe...",Snake,fixed,True,11.942478,24


In [85]:
logger.print_attribution_matrix(exp_id=1,  show_debug_cols=True)

Attribution matrix for experiment 1 
Attribution Strategy: cosine 
Perturbation strategy: fixed:
Input Tokens (Rows) vs. Output Tokens (Columns)


,Snake (0),perturbed_input,perturbed_output
In (0),-0.000000,"J.K. Rowling's Harry Potter series, the spell used to conjure a Patronus is considered highly advanced and can only be performed by skilled witches and wizards. The form that a Patronus takes can vary widely and is often influenced by the caster's personality and experiences. What form does Malfoy's Patronus take? Answer in 1 word.",Snake
J.K. (1),-0.000000,"In Rowling's Harry Potter series, the spell used to conjure a Patronus is considered highly advanced and can only be performed by skilled witches and wizards. The form that a Patronus takes can vary widely and is often influenced by the caster's personality and experiences. What form does Malfoy's Patronus take? Answer in 1 word.",Snake
Rowling's (2),-0.000000,"In J.K. Harry Potter series, the spell used to conjure a Patronus is considered highly advanced and can only be performed by skilled witches and wizards. The form that a Patronus takes can vary widely and is often influenced by the caster's personality and experiences. What form does Malfoy's Patronus take? Answer in 1 word.",Snake
Harry (3),-0.000000,"In J.K. Rowling's Potter series, the spell used to conjure a Patronus is considered highly advanced and can only be performed by skilled witches and wizards. The form that a Patronus takes can vary widely and is often influenced by the caster's personality and experiences. What form does Malfoy's Patronus take? Answer in 1 word.",Snake
Potter (4),-0.000000,"In J.K. Rowling's Harry series, the spell used to conjure a Patronus is considered highly advanced and can only be performed by skilled witches and wizards. The form that a Patronus takes can vary widely and is often influenced by the caster's personality and experiences. What form does Malfoy's Patronus take? Answer in 1 word.",Snake
"series, (5)",-0.000000,In J.K. Rowling's Harry Potter the spell used to conjure a Patronus is considered highly advanced and can only be performed by skilled witches and wizards. The form that a Patronus takes can vary widely and is often influenced by the caster's personality and experiences. What form does Malfoy's Patronus take? Answer in 1 word.,Snake
the (6),-0.000000,"In J.K. Rowling's Harry Potter series, spell used to conjure a Patronus is considered highly advanced and can only be performed by skilled witches and wizards. The form that a Patronus takes can vary widely and is often influenced by the caster's personality and experiences. What form does Malfoy's Patronus take? Answer in 1 word.",Snake
spell (7),-0.000000,"In J.K. Rowling's Harry Potter series, the used to conjure a Patronus is considered highly advanced and can only be performed by skilled witches and wizards. The form that a Patronus takes can vary widely and is often influenced by the caster's personality and experiences. What form does Malfoy's Patronus take? Answer in 1 word.",Snake
used (8),-0.000000,"In J.K. Rowling's Harry Potter series, the spell to conjure a Patronus is considered highly advanced and can only be performed by skilled witches and wizards. The form that a Patronus takes can vary widely and is often influenced by the caster's personality and experiences. What form does Malfoy's Patronus take? Answer in 1 word.",Snake
to (9),-0.000000,"In J.K. Rowling's Harry Potter series, the spell used conjure a Patronus is considered highly advanced and can only be performed by skilled witches and wizards. The form that a Patronus takes can vary widely and is often influenced by the caster's personality and experiences. What form does Malfoy's Patronus take? Answer in 1 word.",Snake
